# Data Extraction, Tranformation and Loading

# 1. Instructional data

In [1]:
import os
import pandas as pd
import requests
import zipfile
from io import BytesIO
from sqlalchemy import create_engine, text

# Define base URL and range of years
base_url = "https://nces.ed.gov/ipeds/datacenter/data/"
years = range(2019, 2024)  # From 2019 to 2023

# Columns to unpivot (if they exist)
columns_to_melt = ["SA_9MCM", "SA09MCM", "SA10MCM", "SA11MCM", "SA12MCM",
        "SA09MOM", "SA10MOM", "SA11MOM", "SA12MOM",
        "SAEQ9OM", "SAEQ9AM", "SA09MAM", "SA10MAM", "SA11MAM", "SA12MAM",
        "SA_9MCW", "SA09MCW", "SA10MCW", "SA11MCW", "SA12MCW",
        "SA09MOW", "SA10MOW", "SA11MOW", "SA12MOW",
        "SAEQ9OW", "SAEQ9AW", "SA09MAW", "SA10MAW", "SA11MAW", "SA12MAW"]

# Create an empty list to store data
all_data = []

# Loop through each year, download, extract, and process the file
for year in years:
    zip_url = f"{base_url}SAL{year}_IS.zip"
    print(f"Downloading {zip_url}...")

    # Download the ZIP file
    response = requests.get(zip_url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content), "r") as z:
            # Find the CSV file inside the ZIP (assuming only one CSV per ZIP)
            csv_filename = [f for f in z.namelist() if f.endswith(".csv")][0]
            print(f"Extracting {csv_filename}...")

            # Read CSV file directly from ZIP
            with z.open(csv_filename) as f:
                df = pd.read_csv(f, dtype=str)  # Ensure all columns are read as strings

            # Add Year column
            df["Year"] = year

            # Only keep columns that exist in the dataset
            available_columns = ["UNITID", "ARANK"] + [col for col in columns_to_melt if col in df.columns]
            df = df[available_columns + ["Year"]]

            # Unpivot data (Convert Wide to Long Format)
            df_melted = df.melt(id_vars=["UNITID", "ARANK", "Year"], 
                                var_name="Item", 
                                value_name="Value")

            # Append to list
            df_melted["Value"] = pd.to_numeric(df_melted["Value"], errors='coerce')
            all_data.append(df_melted)
    else:
        print(f"Failed to download {zip_url}")

# Combine all years into one DataFrame
df = pd.concat(all_data, ignore_index=True)

print(df.columns)  # Check column names

# Drop rows with NaN values in the Value column
final_df = df.dropna(subset=['Value'])

# Database connection details
db_url = "postgresql://rankings:xxxx/rankings?sslmode=require"

# Create a SQLAlchemy engine
engine = create_engine(db_url)

# Delete all content from the existing table
with engine.connect() as connection:
    # Truncate the table
    connection.execute(text("TRUNCATE TABLE instruction;"))
    print("Existing data deleted from instruction table.")

    # Verify that the table is empty
    result = connection.execute(text("SELECT COUNT(*) FROM instruction;"))
    count = result.scalar()
    print(f"Number of rows in instruction table after truncation: {count}")

    if count != 0:
        raise Exception("Truncate command did not execute correctly. Table is not empty.")

# Save DataFrame to PostgreSQL
table_name = "instruction"
final_df.to_sql(table_name, engine, if_exists='append', index=False)
print(f"Data appended to PostgreSQL table {table_name}")

# Verify the number of rows after appending data
with engine.connect() as connection:
    result = connection.execute(text("SELECT COUNT(*) FROM instruction;"))
    count_after_append = result.scalar()
    print(f"Number of rows in instruction table after appending data: {count_after_append}")

Extracting sal2019_is.csv...
Extracting sal2020_is.csv...
Extracting sal2021_is.csv...
Extracting sal2022_is.csv...
Extracting sal2023_is.csv...
Index(['UNITID', 'ARANK', 'Year', 'Item', 'Value'], dtype='object')
Existing data deleted from instruction table.
Number of rows in instruction table after truncation: 0
Data appended to PostgreSQL table instruction
Number of rows in instruction table after appending data: 1865624


# 2.Library data

In [2]:
import os
import pandas as pd
import requests
import zipfile
from io import BytesIO
from sqlalchemy import create_engine, text

# Define base URL and range of years
base_url = "https://nces.ed.gov/ipeds/datacenter/data/"
years = range(2019, 2024)  # From 2019 to 2023

# Columns to unpivot (if they exist)
columns_to_melt = ['LPBOOKS', 'LEBOOKS', 'LEDATAB', 'LPMEDIA', 'LEMEDIA', 'LPSERIA', 'LESERIA', 'LEXOMTL']

# Create an empty list to store data
all_data = []

# Loop through each year, download, extract, and process the file
for year in years:
    zip_url = f"{base_url}AL{year}.zip"
    print(f"Downloading {zip_url}...")

    # Download the ZIP file
    response = requests.get(zip_url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content), "r") as z:
            # Find the CSV file inside the ZIP (assuming only one CSV per ZIP)
            csv_filename = [f for f in z.namelist() if f.endswith(".csv")][0]
            print(f"Extracting {csv_filename}...")

            # Read CSV file directly from ZIP
            with z.open(csv_filename) as f:
                df = pd.read_csv(f, dtype=str)  # Ensure all columns are read as strings

            # Add Year column
            df["Year"] = year

            # Only keep columns that exist in the dataset
            available_columns = ["UNITID", "LCOLELYN"] + [col for col in columns_to_melt if col in df.columns]
            df = df[available_columns + ["Year"]]

            # Unpivot data (Convert Wide to Long Format)
            df_melted = df.melt(id_vars=["UNITID", "LCOLELYN", "Year"], 
                                var_name="Item", 
                                value_name="Value")

            # Append to list
            df_melted["Value"] = pd.to_numeric(df_melted["Value"], errors='coerce')
            all_data.append(df_melted)
    else:
        print(f"Failed to download {zip_url}")

# Combine all years into one DataFrame
df = pd.concat(all_data, ignore_index=True)

print(df.columns)  # Check column names

# Drop rows with NaN values in the Value column
final_df = df.dropna(subset=['Value'])

# Database connection details
db_url = "postgresql://rankings:xxxx/rankings?sslmode=require"

# Create a SQLAlchemy engine
engine = create_engine(db_url)

# Delete all content from the existing table
with engine.connect() as connection:
    # Truncate the table
    connection.execute(text("TRUNCATE TABLE library;"))
    print("Existing data deleted from library table.")

    # Verify that the table is empty
    result = connection.execute(text("SELECT COUNT(*) FROM library;"))
    count = result.scalar()
    print(f"Number of rows in library table after truncation: {count}")

    if count != 0:
        raise Exception("Truncate command did not execute correctly. Table is not empty.")

# Save DataFrame to PostgreSQL
table_name = "library"
final_df.to_sql(table_name, engine, if_exists='append', index=False)
print(f"Data appended to PostgreSQL table {table_name}")

# Verify the number of rows after appending data
with engine.connect() as connection:
    result = connection.execute(text("SELECT COUNT(*) FROM library;"))
    count_after_append = result.scalar()
    print(f"Number of rows in library table after appending data: {count_after_append}")


Extracting al2019.csv...
Extracting al2020.csv...
Extracting al2021.csv...
Extracting al2022.csv...
Extracting al2023.csv...
Index(['UNITID', 'LCOLELYN', 'Year', 'Item', 'Value'], dtype='object')
Existing data deleted from library table.
Number of rows in library table after truncation: 0
Data appended to PostgreSQL table library
Number of rows in library table after appending data: 143336
